todo:
- find the outliers, where the deposit does not equal the payment, all the others can be classified
- asign the transactions to the dkb account so that we can filter them out
    

In [1]:
import sys, os


import logging
import pandas as pd
import numpy as np
from pathlib import Path
# sys.path.insert(0,'..')
project_dir = str(Path(os.getcwd()).parent)
if not project_dir in sys.path:
    sys.path.insert(0,project_dir)
sys.path
from BankClassify import BankClassify, assign_target_account, assign_label
from load_data import read_dkb_csv, read_paypal_csv
from classify_helper import merge_paypal_enties, check_balance
import yaml

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
logging.basicConfig(level=logging.DEBUG)

In [4]:
input_data_path =  Path('/Users/jangie/Documents/Finanzen/Detailed_Data')

In [5]:
# bc.data['dkb_3109'].index = len(bc.data['dkb_3109'])

In [6]:
bc = BankClassify(Path('../data'))

DEBUG:root:FFFFF /Users/jangie/Projects/DataScience/BankClassify/BankClassify.py
INFO:root:loaded previous dkb_3109 data 541 entries
INFO:root:loaded previous paypal data 1017 entries
INFO:root:loaded previous dkb_4217 data 1212 entries
INFO:root:train dataset size 2010


In [7]:
bc.data_all.index.levels[0]

Index(['dkb_3109', 'paypal', 'dkb_4217'], dtype='object')

In [11]:
bc.data_all

datetime   amount  \
dkb_3109 0     2021-12-29 00:00:00   -39.99   
         1     2021-12-27 00:00:00   -57.00   
         2     2021-12-27 00:00:00   -16.79   
         3     2021-12-23 00:00:00  3845.20   
         4     2021-12-22 00:00:00   500.00   
...                            ...      ...   
dkb_4217 1203  2022-01-21 00:00:00   -17.25   
         1204  2022-01-21 00:00:00   -21.95   
         1205  2022-01-20 00:00:00   -40.96   
         1206  2022-01-20 00:00:00    -8.22   
         1211  2022-01-07 00:00:00   -19.50   

                                                            desc  \
dkb_3109 0     Lastschrift Vodafone Deutschland GmbH 12/2021 ...   
         1     Überweisung EDITH ESPINOZA DE THIEKE Rechn.Nr....   
         2     Lastschrift Vodafone GmbH 0000114111198 001508...   
         3     Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...   
         4     Gutschrift DR.ST. U. C. GIESELER Feliz natal p...   
...                                                          ...   
dkb_4217 1203  Kartenzahlung/-abrechnung DM FIL.2814 H:65273/...   
         1204  Kartenzahlung/-abrechnung HUGENDUBEL 0665 FIL....   
         1205  Kartenzahlung/-abrechnung EDEKA ZAHL, BERLIN//...   
         1206  Kartenzahlung/-abrechnung HANSEN OBST GBR GIR ...   
         1211  Lastschrift SWAPFIETS BY BUCKAROO Swapfiets: D...   

              target account                               class  
dkb_3109 0               NaN               Bill - Communications  
         1               NaN       Bill - Utilities, Rent & Fees  
         2               NaN               Bill - Communications  
         3               NaN          Income - Salary & Benefits  
         4               NaN                   Income - Presents  
...                      ...                                 ...  
dkb_4217 1203            NaN  Supermarket & Everyday commodities  
         1204            NaN                               Other  
         1205            NaN  Supermarket & Everyday commodities  
         1206            NaN  Supermarket & Everyday commodities  
         1211            NaN                    Public Transport  

[1856 rows x 5 columns]

In [8]:
for f in Path('../data').glob('*.csv'):
    print(f.with_suffix('').name)

dkb_3109
dkb_4217


In [9]:
# bc.data

In [10]:
sorted([f for f in (input_data_path/'dkb').glob('*.csv')])

[PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2020_1071944217.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2021_1071944217.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1002423109.csv'),
 PosixPath('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/2022_1071944217.csv')]

### load new raw data and add to existing dataset

#### paypal data

In [38]:
filename = Path('/Users/jangie/Documents/Finanzen/Detailed_Data/Paypal/Paypal_01 Jan 2019 - 29 Dec 2022.CSV')
df_pp = read_paypal_csv(filename)
print(f"paypal datasize is {len(df_pp)}")
merge_paypal_enties(df_pp)
print(f"after merge reduced to {len(df_pp)}")
check_balance(df_pp)

# consitency check, merge_paypal_enties, should eliminate all the foreig currency entries
assert (df_pp['Currency'].unique() == ['EUR']).all()

# enties with identical datetime, there should be one that is the transfer to the dkb bank
# if we eliminate these, the remaining datetimes should be unique
assert (df_pp[df_pp['target account'].isna()]['datetime'].value_counts().unique() == [1]).all()

DEBUG:root:loaded 1382 paypal records
DEBUG:root:1084 paypal records after dropping `Balance Impact` = `Memo` entries


paypal datasize is 1084
after merge reduced to 1017
check passed!!


In [39]:
# df_pp[['datetime', 'amount', 'desc', 'target account', 'class']]

In [40]:
bc.add_data(df_pp, "paypal")

DEBUG:root:adding paypal data!
DEBUG:root:previous dataset 1017 entries
DEBUG:root:new dataset 1017 entries with 0 new ones


In [14]:
# bc.data.keys()

In [15]:
# pd.concat(bc.data.values(), keys=bc.data.keys())[['datetime', 'amount', 'desc']]

In [22]:
# bc.data['paypal']

In [25]:
bc.data_unlabeled

datetime  amount  \
paypal 0     2019-05-27 07:02:28+00:00 -100.00   
       1     2020-09-03 08:31:34+00:00    6.00   
       2     2020-09-26 06:53:18+00:00  -98.22   
       4     2020-10-06 03:05:58+00:00  -10.98   
       6     2020-10-09 17:11:33+00:00  -32.70   
...                                ...     ...   
       1007  2022-12-08 18:36:59+00:00   -9.40   
       1009  2022-12-09 00:21:18+00:00  -59.99   
       1011  2022-12-09 10:54:04+00:00  -65.00   
       1013  2022-12-10 18:24:47+00:00  -34.15   
       1015  2022-12-16 12:49:28+00:00  -12.99   

                                                          desc target account  \
paypal 0     Alexander Kihm   Alles Liebe und viel Spass!! ...            NaN   
       1                                     Konrad Exner   �             NaN   
       2     DURSTEXPRESS GmbH Germany Bestellung 10585-247...            NaN   
       4     baby-walz GmbH Germany paypal-payment:62133221...            NaN   
       6                        Takeaway.com Payments B.V.                NaN   
...                                                        ...            ...   
       1007  LogPay Financial Services GmbH  BVG Tickets 4-...            NaN   
       1009                Fashion ID GmbH & Co. KG Germany               NaN   
       1011  CTS EVENTIM Sports GmbH  Tickets Alba Berlin  ...            NaN   
       1013  app smart GmbH / order smart Germany Salmon Lo...            NaN   
       1015                                     Spotify AB                NaN   

            class  
paypal 0      NaN  
       1      NaN  
       2      NaN  
       4      NaN  
       6      NaN  
...           ...  
       1007   NaN  
       1009   NaN  
       1011   NaN  
       1013   NaN  
       1015   NaN  

[532 rows x 5 columns]

#### dkb data

In [43]:
filename = Path('/Users/jangie/Documents/Finanzen/Detailed_Data/dkb/1002423109.csv')

In [44]:
df_new = read_dkb_csv(filename)

INFO:root:number of data loaded: 70
DEBUG:root:new data from /Users/jangie/Documents/Finanzen/Detailed_Data/dkb/1002423109.csv with 70 entries, years [2022]


In [46]:
df_new['date']

0    2022-09-30
1    2022-09-30
2    2022-09-29
3    2022-09-28
4    2022-09-27
        ...    
65   2022-08-04
66   2022-08-03
67   2022-08-03
68   2022-08-02
69   2022-08-01
Name: date, Length: 70, dtype: datetime64[ns]

In [11]:
len(bc.data['dkb_3109'][bc.data['dkb_3109']['class'].isna()])

123

In [12]:
for k, v in bc.data.items():
    print(f"{k}: {len(v)} entires")

dkb_3109: 541 entires
dkb_4217: 1212 entires


In [13]:
df_new

,date,amount,desc,account_nr
0,2022-09-30,-315.64,Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...,1002423109
1,2022-09-30,-4.00,Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...,1002423109
2,2022-09-29,-39.99,Lastschrift Vodafone Deutschland GmbH 09/2022 ...,1002423109
3,2022-09-28,4157.32,"Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...",1002423109
4,2022-09-27,-69.00,Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...,1002423109
...,...,...,...,...
65,2022-08-04,-40.00,Kartenzahlung/-abrechnung RESTAURANT SONNE GIR...,1002423109
66,2022-08-03,-100.00,Lastschrift European Bank for Financial Servic...,1002423109
67,2022-08-03,-23.00,Lastschrift GIRASOLES - SONNENBLUMEN Beitrag N...,1002423109
68,2022-08-02,-7.20,Kartenzahlung/-abrechnung Lagard re Travel Ret...,1002423109


In [14]:
bc.add_data(df_new, f"dkb_{filename.with_suffix('').name[-4:]}")

DEBUG:root:adding dkb_3109 data!
DEBUG:root:previous dataset 541 entries
DEBUG:root:new dataset 70 entries with 0 new ones


In [18]:
bc.data_unlabeled

date    amount  \
dkb_3109 477 2022-09-30   -315.64   
         478 2022-09-30     -4.00   
         479 2022-09-29    -39.99   
         480 2022-09-28   4157.32   
         481 2022-09-27    -69.00   
         483 2022-09-26    -29.84   
         484 2022-09-26    -19.99   
         485 2022-09-22    -48.00   
         486 2022-09-21  -1000.00   
         487 2022-09-21  -1000.00   
         488 2022-09-20    -90.00   
         489 2022-09-19    -48.00   
         490 2022-09-19    -42.80   
         491 2022-09-19    -18.59   
         492 2022-09-19    -11.94   
         493 2022-09-19     -5.00   
         495 2022-09-12    -34.00   
         496 2022-09-12    -26.00   
         497 2022-09-09    -48.00   
         498 2022-09-09     -5.69   
         499 2022-09-08     -3.95   
         500 2022-09-08     -1.44   
         501 2022-09-07    -65.95   
         502 2022-09-06   -100.00   
         503 2022-09-06    -61.00   
         504 2022-09-06    -22.59   
         505 2022-09-06    -12.60   
         506 2022-09-06     -4.90   
         507 2022-09-05   -100.00   
         508 2022-09-05    -23.00   
         509 2022-09-01  -2450.50   
         510 2022-09-01    -48.00   
         511 2022-09-01    -48.00   
         512 2022-08-31 -20000.00   
         513 2022-08-30 -20000.00   
         514 2022-08-30    -39.00   
         515 2022-08-30     -9.30   
         516 2022-08-30   3983.32   
         517 2022-08-29    -39.99   
         518 2022-08-29     -7.00   
         519 2022-08-26 -25000.00   
         520 2022-08-25    -19.99   
         522 2022-08-24    -38.00   
         523 2022-08-24     -3.80   
         525 2022-08-22  -1000.00   
         526 2022-08-22  -1000.00   
         527 2022-08-22    -90.00   
         528 2022-08-22    -56.58   
         529 2022-08-22    -48.00   
         531 2022-08-19    150.00   
         532 2022-08-18     -2.44   
         533 2022-08-16    -17.80   
         534 2022-08-15    -55.08   
         535 2022-08-15    -48.00   
         536 2022-08-15    -29.05   
         539 2022-08-08   -116.01   
         540 2022-08-08    -61.00   

                                                           desc  account_nr  \
dkb_3109 477  Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...  1002423109   
         478  Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...  1002423109   
         479  Lastschrift Vodafone Deutschland GmbH 09/2022 ...  1002423109   
         480  Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...  1002423109   
         481  Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...  1002423109   
         483  Kartenzahlung/-abrechnung EDEKA GRACKA, BERLIN...  1002423109   
         484  Lastschrift Vodafone GmbH 0000114111198 001835...  1002423109   
         485  Überweisung GISELA SILVA Rechnungs-Nr.: 138/20...  1002423109   
         486  Lastschrift Sauren SICAV SPARPLAN ISIN LU01155...  1002423109   
         487  Lastschrift Sauren SICAV SPARPLAN ISIN LU17170...  1002423109   
         488  Dauerauftrag GIRASOLES-FOERDERVEREIN E. V.  I....  1002423109   
         489  Überweisung GISELA SILVA Rechnungs-Nr.: 133/20...  1002423109   
         490  Kartenzahlung/-abrechnung ALDI SAGT DANKE 28 0...  1002423109   
         491  Kartenzahlung/-abrechnung ASIL GMBH//BERLIN/DE...  1002423109   
         492  Kartenzahlung/-abrechnung Atlantis-Apotheke//B...  1002423109   
         493  Kartenzahlung/-abrechnung BONG APOTHEKE//BERLI...  1002423109   
         495  Kartenzahlung/-abrechnung VINOTRIA GMBH CO KG/...  1002423109   
         496  Kartenzahlung/-abrechnung BEZIRKSAMT CHARLOTTE...  1002423109   
         497  Überweisung GISELA SILVA Rechnungs-Nr.: 129/20...  1002423109   
         498  Kartenzahlung/-abrechnung Burger King 15800//N...  1002423109   
         499  Kartenzahlung/-abrechnung Der Beck GmbH Filial...  1002423109   
         500  Kartenzahlung/-abrechnung MUELLER SAGT DANKE//...  1002423109   
         501  Kartenzahlung/-abrechnung DECATHLON Deuts

### assign target account

In [16]:
r'PAYPAL (EUROPE) S.A.R.L .*'

'PAYPAL (EUROPE) S.A.R.L .*'

In [17]:
df = bc.data_unlabeled

In [18]:
# df[df['desc'].str.match(r'^Gutschrift J.*und C.*')]

In [19]:
# bc.data_unlabeled[bc.data_unlabeled['amount']> 2000]

In [20]:
df_update = assign_target_account(bc.data_unlabeled)
df_update

DEBUG:root:assigning 5 entries to dkb_4217


>>>>PAYPAL (EUROPE) S.A.R.L .*
>>>>^Überweisung HERR .*
>>>>Umbuchung KREDITKARTEN GELDANLAGE
>>>>^Gutschrift J.*und C.*


date   amount  \
dkb_3109 477 2022-09-30  -315.64   
         478 2022-09-30    -4.00   
         479 2022-09-29   -39.99   
         480 2022-09-28  4157.32   
         481 2022-09-27   -69.00   
...                 ...      ...   
         534 2022-08-15   -55.08   
         535 2022-08-15   -48.00   
         536 2022-08-15   -29.05   
         539 2022-08-08  -116.01   
         540 2022-08-08   -61.00   

                                                           desc  account_nr  \
dkb_3109 477  Kartenzahlung/-abrechnung VIVA RESORT PROT.PRI...  1002423109   
         478  Kartenzahlung/-abrechnung H.RESORT SUPERMERCAD...  1002423109   
         479  Lastschrift Vodafone Deutschland GmbH 09/2022 ...  1002423109   
         480  Lohn, Gehalt, Rente IAV GmbH Ingenieurgesellsc...  1002423109   
         481  Überweisung GISELA SILVA Rechnungs-Nr.: 142/20...  1002423109   
...                                                         ...         ...   
         534  Lastschrift Rundfunk ARD, ZDF, DRadio Rundfunk...  1002423109   
         535  Überweisung GISELA SILVA Rechnungs-Nr.: 100/20...  1002423109   
         536  Kartenzahlung/-abrechnung ROSSMANN VIELEN DANK...  1002423109   
         539  Kartenzahlung/-abrechnung AMBROSINO SUPERMARKT...  1002423109   
         540  Lastschrift VATTENFALL EUROPE SALES S/83663436...  1002423109   

             target account class  
dkb_3109 477            NaN   NaN  
         478            NaN   NaN  
         479            NaN   NaN  
         480            NaN   NaN  
         481            NaN   NaN  
...                     ...   ...  
         534            NaN   NaN  
         535            NaN   NaN  
         536            NaN   NaN  
         539            NaN   NaN  
         540            NaN   NaN  

[62 rows x 6 columns]

In [21]:
df_update[~df_update['target account'].isna()]

date   amount  \
dkb_3109 482 2022-09-27   5000.0   
         494 2022-09-19   3000.0   
         521 2022-08-25  20000.0   
         524 2022-08-23  20000.0   
         530 2022-08-22  20000.0   

                                                           desc  account_nr  \
dkb_3109 482  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         494  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         521  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         524  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   
         530  Gutschrift Jan Gieseler und Camila Lobo Costa ...  1002423109   

             target account class  
dkb_3109 482       dkb_4217   NaN  
         494       dkb_4217   NaN  
         521       dkb_4217   NaN  
         524       dkb_4217   NaN  
         530       dkb_4217   NaN

In [22]:
# df_update.loc['dkb_3109']

In [23]:
bc.data.keys()

dict_keys(['dkb_3109', 'dkb_4217'])

In [24]:
bc.update_data(df_update)

DEBUG:root:updating dataset_name: dkb_3109


ffff Index(['date', 'amount', 'desc', 'account_nr', 'target account', 'class'], dtype='object')


In [29]:
len(bc.data_unlabeled)

57

In [30]:
bc.save_data()

### label that data with regex

In [23]:
bc.data_unlabeled['desc'].value_counts()

REWE Markt GmbH Germany Ihre Bestellung bei REWE (Lieferservice)  Ihre Bestellung bei REWE (Lieferservice)    28
nextbike GmbH  Overusage / negative balance  Overusage / negative balance                                     13
Berliner Bäder-Betriebe Germany                                                                               12
Fashion Retail SA Germany                                                                                     11
Takeaway.com Payments B.V.                                                                                    10
                                                                                                              ..
Google  Mare of Easttown S1 • E1  Mare of Easttown S1 • E1                                                     1
Google  Mare of Easttown S1 • E2  Mare of Easttown S1 • E2                                                     1
Google  Mare of Easttown S1 • E3  Mare of Easttown S1 • E3                                      

In [20]:
bc.data_unlabeled

datetime  amount  \
paypal 0     2019-05-27 07:02:28+00:00 -100.00   
       4     2020-10-06 03:05:58+00:00  -10.98   
       6     2020-10-09 17:11:33+00:00  -32.70   
       10    2020-10-15 10:24:14+00:00 -127.68   
       12    2020-10-15 18:46:05+00:00  -10.49   
...                                ...     ...   
       983   2022-11-23 00:17:17+00:00 -108.94   
       989   2022-11-25 21:03:21+00:00  -52.80   
       999   2022-12-05 13:20:00+00:00 -163.00   
       1009  2022-12-09 00:21:18+00:00  -59.99   
       1011  2022-12-09 10:54:04+00:00  -65.00   

                                                          desc target account  \
paypal 0     Alexander Kihm   Alles Liebe und viel Spass!! ...            NaN   
       4     baby-walz GmbH Germany paypal-payment:62133221...            NaN   
       6                        Takeaway.com Payments B.V.                NaN   
       10    Conrad Electronic SE Germany CE 001500207CE 00...            NaN   
       12    buecher.de GmbH & Co. KG Germany Ihre Bestellu...            NaN   
...                                                        ...            ...   
       983   Nike Retail BV Germany Nike Order  Nike Swoosh...            NaN   
       989       Caudalie SAS Germany 72000462985  72000462985            NaN   
       999   SCC EVENTS GmbH  ORD-0524832. BMW BERLIN-MARAT...            NaN   
       1009                Fashion ID GmbH & Co. KG Germany               NaN   
       1011  CTS EVENTIM Sports GmbH  Tickets Alba Berlin  ...            NaN   

            class  
paypal 0      NaN  
       4      NaN  
       6      NaN  
       10     NaN  
       12     NaN  
...           ...  
       983    NaN  
       989    NaN  
       999    NaN  
       1009   NaN  
       1011   NaN  

[331 rows x 5 columns]

In [21]:
df_update.loc['paypal', 12]['desc']

'buecher.de GmbH & Co. KG Germany Ihre Bestellung bei bücher.de  Ihre Bestellung bei bücher.de'

In [22]:
df_update = assign_label(bc.data_unlabeled)
print(len(df_update))
df_update

331


datetime  amount  \
paypal 0     2019-05-27 07:02:28+00:00 -100.00   
       4     2020-10-06 03:05:58+00:00  -10.98   
       6     2020-10-09 17:11:33+00:00  -32.70   
       10    2020-10-15 10:24:14+00:00 -127.68   
       12    2020-10-15 18:46:05+00:00  -10.49   
...                                ...     ...   
       983   2022-11-23 00:17:17+00:00 -108.94   
       989   2022-11-25 21:03:21+00:00  -52.80   
       999   2022-12-05 13:20:00+00:00 -163.00   
       1009  2022-12-09 00:21:18+00:00  -59.99   
       1011  2022-12-09 10:54:04+00:00  -65.00   

                                                          desc target account  \
paypal 0     Alexander Kihm   Alles Liebe und viel Spass!! ...            NaN   
       4     baby-walz GmbH Germany paypal-payment:62133221...            NaN   
       6                        Takeaway.com Payments B.V.                NaN   
       10    Conrad Electronic SE Germany CE 001500207CE 00...            NaN   
       12    buecher.de GmbH & Co. KG Germany Ihre Bestellu...            NaN   
...                                                        ...            ...   
       983   Nike Retail BV Germany Nike Order  Nike Swoosh...            NaN   
       989       Caudalie SAS Germany 72000462985  72000462985            NaN   
       999   SCC EVENTS GmbH  ORD-0524832. BMW BERLIN-MARAT...            NaN   
       1009                Fashion ID GmbH & Co. KG Germany               NaN   
       1011  CTS EVENTIM Sports GmbH  Tickets Alba Berlin  ...            NaN   

            class  
paypal 0      NaN  
       4      NaN  
       6      NaN  
       10     NaN  
       12     NaN  
...           ...  
       983    NaN  
       989    NaN  
       999    NaN  
       1009   NaN  
       1011   NaN  

[331 rows x 5 columns]

In [30]:
bc.update_data(df_update)

DEBUG:root:updating dataset_name: paypal


ffff Index(['datetime', 'amount', 'desc', 'target account', 'class'], dtype='object')
>> ['datetime', 'amount', 'desc', 'target account', 'class', 'Type', 'From Email Address', 'To Email Address', 'Currency', 'Status', 'Balance', 'Transaction ID']
{'desc', 'target account', 'class', 'amount', 'datetime'}
{'To Email Address', 'desc', 'Balance', 'target account', 'class', 'Type', 'Currency', 'Transaction ID', 'amount', 'Status', 'From Email Address', 'datetime'}
True


In [36]:
bc.save_data()

In [13]:
bc.data_unlabeled

datetime  amount  \
paypal 0     2019-05-27 07:02:28+00:00 -100.00   
       1     2020-09-03 08:31:34+00:00    6.00   
       4     2020-10-06 03:05:58+00:00  -10.98   
       6     2020-10-09 17:11:33+00:00  -32.70   
       8     2020-10-09 19:17:41+00:00  -10.00   
...                                ...     ...   
       1003  2022-12-06 15:28:09+00:00   -6.00   
       1005  2022-12-07 20:48:25+00:00  -32.00   
       1007  2022-12-08 18:36:59+00:00   -9.40   
       1009  2022-12-09 00:21:18+00:00  -59.99   
       1011  2022-12-09 10:54:04+00:00  -65.00   

                                                          desc target account  \
paypal 0     Alexander Kihm   Alles Liebe und viel Spass!! ...            NaN   
       1                                     Konrad Exner   �             NaN   
       4     baby-walz GmbH Germany paypal-payment:62133221...            NaN   
       6                        Takeaway.com Payments B.V.                NaN   
       8     Vodafone GmbH  15237806204-337858847-10EUR-PP-...            NaN   
...                                                        ...            ...   
       1003  LogPay Financial Services GmbH  BVG Tickets 4-...            NaN   
       1005             Hansemerkur Krankenversicherung AG                NaN   
       1007  LogPay Financial Services GmbH  BVG Tickets 4-...            NaN   
       1009                Fashion ID GmbH & Co. KG Germany               NaN   
       1011  CTS EVENTIM Sports GmbH  Tickets Alba Berlin  ...            NaN   

            class  
paypal 0      NaN  
       1      NaN  
       4      NaN  
       6      NaN  
       8      NaN  
...           ...  
       1003   NaN  
       1005   NaN  
       1007   NaN  
       1009   NaN  
       1011   NaN  

[491 rows x 5 columns]

### define the regex expressions and save to disk

In [12]:
regex_dict = {
    r'^Lastschrift Vodafone .*': 'Bill - Communications',
    r'^Vodafone GmbH .*': 'Bill - Communications',
    r'^Lohn, Gehalt, Rente IAV GmbH *.': 'Income - Salary & Benefits',
    r'^Lastschrift European Bank for Financial Services *.':'Investments',
    r'^Lastschrift Sauren SICAV SPARPLAN *.':'Investments',
    r'^Überweisung GISELA SILVA *.':'Education',
    r'^Dauerauftrag GIRASOLES-FOERDERVEREIN *.':'Education',
    r'^Lastschrift GIRASOLES - SONNENBLUMEN Beitrag .*':'Education',
    r'^buecher.de GmbH & Co. KG*': 'Education',
    r'^Lastschrift VATTENFALL EUROPE *.': 'Bill - Utilities, Rent & Fees',
    r'^Lastschrift Aberdeen Standard Investments *.':'Bill - Utilities, Rent & Fees',
    r'^Kartenzahlung/-abrechnung BARES GbR *.':'Restaurants & Bars',
    r'^app smart GmbH / order smart Germany Salmon Lo.*':'Restaurants & Bars',
    r'^Takeaway.com Payments B.V.':'Restaurants & Bars',
    r'^Kartenzahlung/-abrechnung EDEKA *.': 'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung *.ROSSMANN *.':'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung MUELLER*.': 'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung ALDI SAGT DANKE *.' : 'Supermarket & Everyday commodities',
    r'^Kartenzahlung/-abrechnung DENN?S BIOMARKT *.' : 'Supermarket & Everyday commodities',
    r'^REWE Markt GmbH Germany Ihre Bestellung bei REWE .*': 'Supermarket & Everyday commodities',
    r'^DURSTEXPRESS GmbH Germany': 'Supermarket & Everyday commodities',
    r'.*apotheke.*': 'Personal Care & Sport',
    r'.*Spotify.*': 'Entertainment, Hobbies & Memberships',
    r'^Berliner Bäder-Betriebe Germany.*': 'Entertainment, Hobbies & Memberships',
    r'^LogPay Financial Services GmbH  BVG Tickets .*':'Public Transport',
    r'nextbike GmbH .*':'Public Transport',
    r'^Hansemerkur Krankenversicherung AG.*':'Bill - Insurrance',
    r'^Fashion Retail SA Germany.*': 'Clothes & Shopping',
}


with open(Path('../data/labeling_regex.yaml'), 'w') as file:
    documents = yaml.dump(regex_dict, file)




In [57]:
with open(Path('../data/labeling_regex.yaml')) as file:
    documents = yaml.full_load(file)

In [58]:
documents

{'.*Spotify.*': 'Entertainment, Hobbies & Memberships',
 '.*apotheke.*': 'Personal Care & Sport',
 '^Dauerauftrag GIRASOLES-FOERDERVEREIN *.': 'Education',
 '^Kartenzahlung/-abrechnung *.ROSSMANN *.': 'Supermarket & Everyday commodities',
 '^Kartenzahlung/-abrechnung ALDI SAGT DANKE *.': 'Supermarket & Everyday commodities',
 '^Kartenzahlung/-abrechnung BARES GbR *.': 'Restaurants & Bars',
 '^Kartenzahlung/-abrechnung DENN?S BIOMARKT *.': 'Supermarket & Everyday commodities',
 '^Kartenzahlung/-abrechnung EDEKA *.': 'Supermarket & Everyday commodities',
 '^Kartenzahlung/-abrechnung MUELLER*.': 'Supermarket & Everyday commodities',
 '^Lastschrift Aberdeen Standard Investments *.': 'Bill - Utilities, Rent & Fees',
 '^Lastschrift European Bank for Financial Services *.': 'Investments',
 '^Lastschrift GIRASOLES - SONNENBLUMEN Beitrag .*': 'Education',
 '^Lastschrift Sauren SICAV SPARPLAN *.': 'Investments',
 '^Lastschrift VATTENFALL EUROPE *.': 'Bill - Utilities, Rent & Fees',
 '^Lastschrif

In [39]:
target_dict = {
    r'PAYPAL (EUROPE) S.A.R.L .*': 'paypal', # filter out the paypal entries
    r'^Überweisung HERR .*': 'dkb',
    r'Umbuchung KREDITKARTEN GELDANLAGE': 'dkb',
    r'^Gutschrift J.*und C.*': 'dkb_4217',
    r'^Überweisung HERR DR.*AN.*': 'dkb_3109'
}
with open(Path('../data/target_account_regex.yaml'), 'w') as file:
    documents = yaml.dump(target_dict, file)

In [40]:
with open(Path('../data/target_account_regex.yaml')) as file:
    documents = yaml.full_load(file)
documents

{'PAYPAL (EUROPE) S.A.R.L .*': 'paypal',
 'Umbuchung KREDITKARTEN GELDANLAGE': 'dkb',
 '^Gutschrift J.*und C.*': 'dkb_4217',
 '^Überweisung HERR .*': 'dkb',
 '^Überweisung HERR DR.*AN.*': 'dkb_3109'}

### manually label data with regex

In [62]:
bc.data_all[bc.data_all['desc'].str.lower().str.match(r'^app smart GmbH / order smart Germany Salmon Lo.*'.lower())]


datetime  amount  \
paypal 337   2021-07-22 18:19:02+00:00  -34.20   
       699   2022-04-29 18:31:47+00:00  -31.80   
       719   2022-05-06 18:21:31+00:00  -43.00   
       741   2022-05-20 17:20:36+00:00  -31.30   
       787   2022-06-24 17:49:55+00:00  -35.70   
       797   2022-07-02 18:04:33+00:00  -37.10   
       819   2022-07-15 17:58:34+00:00  -31.70   
       837   2022-07-22 18:15:14+00:00  -36.70   
       851   2022-08-12 18:28:55+00:00  -27.70   
       862   2022-08-19 17:35:08+00:00  -31.50   
       872   2022-09-02 18:38:59+00:00  -35.50   
       891   2022-09-17 17:22:10+00:00  -31.30   
       895   2022-09-23 17:32:59+00:00  -33.80   
       915   2022-10-07 15:58:45+00:00  -31.30   
       963   2022-11-11 18:00:19+00:00  -35.30   
       965   2022-11-11 19:16:56+00:00  -35.30   
       973   2022-11-18 18:20:35+00:00  -32.50   
       1013  2022-12-10 18:24:47+00:00  -34.15   

                                                          desc  account_nr  \
paypal 337   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       699   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       719   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       741   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       787   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       797   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       819   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       837   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       851   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       862   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       872   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       891   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       895   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       915   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       963   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       965   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       973   app smart GmbH / order smart Germany Salmon Lo...         NaN   
       1013  app smart GmbH / order smart Germany Salmon Lo...         NaN   

            target account class                      Type From Email Address  \
paypal 337             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       699             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       719             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       741             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       787             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       797             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       819             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       837             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       851             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       862             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       872             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       891             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       895             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       915             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       963             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       965             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       973             NaN   NaN  Express Checkout Payment       jangie@pm.me   
       1013            NaN   NaN  Express Checkout Payment       jangie@pm.me   

                To Email Address Currency     Status  Ba

In [32]:
len(bc.data_all)

1324

### manually label

In [18]:
i = bc.data_unlabeled[bc.data_unlabeled['amount']>0].index

In [32]:
bc.data['paypal'].loc[bc.data['paypal']['amount']>0, 'class']= 'Income - Reimbursement'

In [34]:
bc.data['paypal'].loc[bc.data['paypal']['amount']>0, 'class']

1       Income - Reimbursement
3       Income - Reimbursement
5       Income - Reimbursement
7       Income - Reimbursement
9       Income - Reimbursement
                 ...          
1008    Income - Reimbursement
1010    Income - Reimbursement
1012    Income - Reimbursement
1014    Income - Reimbursement
1016    Income - Reimbursement
Name: class, Length: 513, dtype: object

In [24]:
data['paypal'].loc[i, ['class']] = 'Income - Reimbursement'

In [35]:
bc.data_unlabeled

datetime  amount  \
paypal 0     2019-05-27 07:02:28+00:00 -100.00   
       4     2020-10-06 03:05:58+00:00  -10.98   
       6     2020-10-09 17:11:33+00:00  -32.70   
       10    2020-10-15 10:24:14+00:00 -127.68   
       12    2020-10-15 18:46:05+00:00  -10.49   
...                                ...     ...   
       983   2022-11-23 00:17:17+00:00 -108.94   
       989   2022-11-25 21:03:21+00:00  -52.80   
       999   2022-12-05 13:20:00+00:00 -163.00   
       1009  2022-12-09 00:21:18+00:00  -59.99   
       1011  2022-12-09 10:54:04+00:00  -65.00   

                                                          desc target account  \
paypal 0     Alexander Kihm   Alles Liebe und viel Spass!! ...            NaN   
       4     baby-walz GmbH Germany paypal-payment:62133221...            NaN   
       6                        Takeaway.com Payments B.V.                NaN   
       10    Conrad Electronic SE Germany CE 001500207CE 00...            NaN   
       12    buecher.de GmbH & Co. KG Germany Ihre Bestellu...            NaN   
...                                                        ...            ...   
       983   Nike Retail BV Germany Nike Order  Nike Swoosh...            NaN   
       989       Caudalie SAS Germany 72000462985  72000462985            NaN   
       999   SCC EVENTS GmbH  ORD-0524832. BMW BERLIN-MARAT...            NaN   
       1009                Fashion ID GmbH & Co. KG Germany               NaN   
       1011  CTS EVENTIM Sports GmbH  Tickets Alba Berlin  ...            NaN   

            class  
paypal 0      NaN  
       4      NaN  
       6      NaN  
       10     NaN  
       12     NaN  
...           ...  
       983    NaN  
       989    NaN  
       999    NaN  
       1009   NaN  
       1011   NaN  

[331 rows x 5 columns]

In [37]:
bc.save_data()

SyntaxError: unexpected EOF while parsing (23938876.py, line 1)